# **<center> CHE1125 - Homework 5 </center>**

### **<center> Abhishek Patel </center>**

In [1]:
import cobra
import numpy as np

In [2]:
# Importing all the models
e_coli_core = cobra.io.load_model('e_coli_core')
iJR904 = cobra.io.load_model('iJR904')
iAF1260 = cobra.io.load_model('iAF1260')
iJO1366 = cobra.io.load_model('iJO1366')
iML1515 = cobra.io.load_model('iML1515')

# 1. Identify the number of lethal reaction deletions in the following models of E.coli’s metabolism:

## a. E.coli core

In [3]:
e_coli_core_deletion_result = cobra.flux_analysis.single_reaction_deletion(e_coli_core)
e_coli_core_lethal_deletions = e_coli_core_deletion_result[e_coli_core_deletion_result['growth'] < 1e-5]
print("No. of essential reactions in the e_coli_core model: ",len(e_coli_core_lethal_deletions) - 1)

No. of essential reactions in the e_coli_core model:  15


## b. iJR904

In [4]:
iJR904_deletion_result = cobra.flux_analysis.single_reaction_deletion(iJR904)
iJR904_lethal_deletions = iJR904_deletion_result[iJR904_deletion_result['growth'] < 1e-5]
print("No. of essential reactions in the iJR904 model: ",len(iJR904_lethal_deletions) - 1)

No. of essential reactions in the iJR904 model:  216


## c. iAF1260

In [5]:
iAF1260_deletion_result = cobra.flux_analysis.single_reaction_deletion(iAF1260)
iAF1260_lethal_deletions = iAF1260_deletion_result[iAF1260_deletion_result['growth'] < 1e-5]
print("No. of essential reactions in the iAF1260 model: ",len(iAF1260_lethal_deletions) - 1)

No. of essential reactions in the iAF1260 model:  278


## d. iJO1366

In [6]:
iJO1366_deletion_result = cobra.flux_analysis.single_reaction_deletion(iJO1366)
iJO1366_lethal_deletions = iJO1366_deletion_result[iJO1366_deletion_result['growth'] < 1e-5]
print("No. of essential reactions in the iJO1366 model: ",len(iJO1366_lethal_deletions) - 1)

No. of essential reactions in the iJO1366 model:  287


## e. iML1515

In [7]:
iML1515_deletion_result = cobra.flux_analysis.single_reaction_deletion(iML1515)
iML1515_lethal_deletions = iML1515_deletion_result[iML1515_deletion_result['growth'] < 1e-5]
print("No. of essential reactions in the iML1515 model: ",len(iML1515_lethal_deletions) - 1)

No. of essential reactions in the iML1515 model:  269


# 2. Write your own algorithm/function to accept a metabolic model and perform FVA. 

The steps for FVA are:

a. Optimize the model for maximum flux through the biomass reaction (maximize growth rate).

b. Constrain the flux bounds of the biomass reaction at its maximum value.

c. For each reaction in the model that is not the biomass reaction:
-  Set the reaction as the model's objective.
-  Optimize the model to predict the maximum flux through that reaction and store it somewhere
-  Optimize the model to predict the minimum flux through that reaction and store it somewhere

**Hints:**

**`1. You will need to use a "for" loop.`**

**`2. cobrapy comes with a function called optimize(). It can be called using a model object as follows:`**

- model.optimize(objective_sense='maximize')
- model.optimize(objective_sense='minimize')

**`The argument 'objective_sense' can be used to specify if you want the solver to maximize or minimize the objective function. This precludes the need to change the objective coefficient to -1 to minimize the reaction. If no objective_sense is specified in the function call, the solver will use the last used objective_sense value or by default, maximize.`**

**`3. Using the 'with' statement within the for loop, could help make your code more concise.`**

**`4. Write code for the first few reactions in the model and then expand it to all reactions in the model. This will help reduce run-time and therefore, the time required to debug your program.`**

In [8]:
def perform_fva(model, biomass_reaction_id):
#     Step a: Optimize for maximum flux through the biomass reaction
    model.objective = model.reactions.get_by_id(biomass_reaction_id)
    model.optimize(objective_sense='maximize')
    max_biomass_flux = model.objective.value

#     Step b: Constrain the flux bounds of the biomass reaction at its maximum value
    model.reactions.get_by_id(biomass_reaction_id).bounds = (max_biomass_flux, max_biomass_flux)

#     Step c: Perform FVA for each reaction in the model
    fva_results = {}
    for reaction in model.reactions:
        if reaction.id != biomass_reaction_id:
#             Set the reaction as the model's objective
            with model as submodel:
                submodel.objective = reaction

#                 Optimize to predict the maximum flux through the reaction
                submodel.optimize(objective_sense='maximize')
                max_flux = submodel.objective.value

#                 Optimize to predict the minimum flux through the reaction
                submodel.optimize(objective_sense='minimize')
                min_flux = submodel.objective.value

#             Store the FVA results for the reaction
            fva_results[reaction.id] = (min_flux, max_flux)

    return fva_results

In [9]:
# Using the 'iJO1366' model and the its biomass reaction identifier
iJO1366 = cobra.io.load_model('iJO1366')
biomass_reaction_id = 'BIOMASS_Ec_iJO1366_core_53p95M'  # Replace with the actual identifier
fva_results = perform_fva(iJO1366, biomass_reaction_id)

In [17]:
# FVA results for each reaction
for reaction_id, (min_flux, max_flux) in fva_results.items():
    print(f'Reaction: {reaction_id}, Min Flux: {min_flux}, Max Flux: {max_flux}')

Reaction: EX_cm_e, Min Flux: 0.0, Max Flux: 0.0
Reaction: EX_cmp_e, Min Flux: 0.0, Max Flux: 0.0
Reaction: EX_co2_e, Min Flux: 19.675222635662806, Max Flux: 19.675222635663292
Reaction: EX_cobalt2_e, Min Flux: -2.4559295318174615e-05, Max Flux: -2.455929531817436e-05
Reaction: DM_4crsol_c, Min Flux: 0.00021906891423811525, Max Flux: 0.00021906891423811647
Reaction: DM_5drib_c, Min Flux: 0.00022103365786356924, Max Flux: 0.0002210336579948262
Reaction: DM_aacald_c, Min Flux: 0.0, Max Flux: 0.0
Reaction: DM_amob_c, Min Flux: 1.964743625453948e-06, Max Flux: 1.964743625453968e-06
Reaction: DM_mththf_c, Min Flux: 0.0004401025721069751, Max Flux: 0.0004401025722001481
Reaction: EX_colipa_e, Min Flux: -3.907732907465533e-17, Max Flux: 1.8577910802814102e-14
Reaction: DM_oxam_c, Min Flux: 0.0, Max Flux: -2.2003145129038888e-13
Reaction: EX_glc__D_e, Min Flux: -10.0, Max Flux: -9.999999999999893
Reaction: EX_glcn_e, Min Flux: 0.0, Max Flux: 1.1185066480901224e-13
Reaction: BIOMASS_Ec_iJO1366_W

# 3.Using your algorithm, calculate the sum of ranges of fluxes in the iJO1366 model. Compare this with that obtained from the in-built FVA function in cameo.

In [11]:
# Sum of the ranges of fluxes for all reactions using algorithm created by me.
flux_ranges = [max_flux - min_flux for min_flux, max_flux in fva_results.values()]
sum_of_flux_ranges = sum(flux_ranges)

print(f"Sum of the ranges of fluxes in the iJO1366 model using Algorithm 1: {sum_of_flux_ranges}")

Sum of the ranges of fluxes in the iJO1366 model using Algorithm 1: 89942.168245081


In [12]:
fva_results_2 = cobra.flux_analysis.flux_variability_analysis(iJO1366, iJO1366.reactions, fraction_of_optimum = 1)
ranges = abs(fva_results_2['minimum'] - fva_results_2['maximum'])
print(f"Sum of the ranges of fluxes in the iJO1366 model using In-built Algorithm: {sum(ranges)}")

Sum of the ranges of fluxes in the iJO1366 model using In-built Algorithm: 89942.1677480303
